In [75]:
import os
import torch
import torchaudio

from encodec import EncodecModel
from encodec.utils import convert_audio, save_audio

In [76]:
input_dir = "./samples/inputs/"
output_dir = "./samples/outputs/"
sample_rate = 24000
device = "cuda" if torch.cuda.is_available() else "cpu"

In [83]:
model = EncodecModel._get_model(target_bandwidths=[1.5, 3, 6, 12, 24], 
                                sample_rate=sample_rate, 
                                channels=1, causal=False, audio_normalize=False, segment=None).to(device)

In [84]:
model.load_state_dict(torch.load("./training/model_two.pth"))

C:\Users\debja\AppData\Local\Temp\ipykernel_9904\3267540992.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("./training/model_two.pth"))

<All keys matched successfully>

In [79]:
def sample(audio_file_path, out_path, model):
    wv, _ = torchaudio.load(audio_file_path)
    if len(wv.shape) < 3:
        wv = wv.unsqueeze(0).to(device)
    model.eval()
    out_wv = model(wv)[0].detach().cpu()
    out_wv = convert_audio(out_wv, sample_rate, sample_rate, target_channels=1)
    save_audio(out_wv, out_path, sample_rate)

In [85]:
for f in os.listdir(input_dir):
    audio_file_path = os.path.join(input_dir, f)
    fname, ext = os.path.splitext(os.path.basename(audio_file_path))
    fout_name = f"{fname}_out{ext}"
    fout_path = os.path.join(output_dir, fout_name)
    sample(audio_file_path, fout_path, model)